In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import sqlite3 # library for working with sqlite database
conn = sqlite3.connect("./data/data.db") # Create a connection to the on-disk database

In [2]:
pd.read_sql("SELECT * FROM sqlite_master where type='table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,patients,patients,2,"CREATE TABLE patients(ROW_ID INT,SUBJECT_ID IN..."
1,table,admissions,admissions,639,"CREATE TABLE admissions(ROW_ID INT,SUBJECT_ID ..."
2,table,icustays,icustays,3515,"CREATE TABLE icustays(ROW_ID INT,SUBJECT_ID IN..."
3,table,diagnoses_icd,diagnoses_icd,4944,"CREATE TABLE diagnoses_icd(ROW_ID INT,SUBJECT_..."
4,table,drgcodes,drgcodes,9061,"CREATE TABLE drgcodes(ROW_ID INT,SUBJECT_ID IN..."


In [3]:
pd.read_sql("""SELECT * FROM admissions LIMIT 10""", conn)


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,,CATHOLIC,MARRIED,WHITE,,,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,,,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,,PROTESTANT QUAKER,SINGLE,WHITE,,,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,,,V-TACH,0,1
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,,CATHOLIC,,WHITE,,,NEWBORN,0,1
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,,CATHOLIC,MARRIED,WHITE,,,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,,,UNSTABLE ANGINA\CATH,0,1
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,,CATHOLIC,MARRIED,WHITE,,,STATUS EPILEPTICUS,1,1


In [4]:
pat_df = pd.read_sql("SELECT * FROM patients", conn)
adm_df = pd.read_sql("SELECT * FROM admissions", conn)
pat_df.shape[0]

46520

In [5]:
pat_df.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,,,,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,,1
2,236,251,M,2090-03-15 00:00:00,,,,0
3,237,252,M,2078-03-06 00:00:00,,,,0
4,238,253,F,2089-11-26 00:00:00,,,,0


In [6]:
all_admissions = pd.read_sql("""SELECT * FROM admissions""", conn)
print(all_admissions.shape)
all_admissions = all_admissions.sort_values('SUBJECT_ID')
all_admissions_info = all_admissions[['SUBJECT_ID', 'HADM_ID', 'HOSPITAL_EXPIRE_FLAG']]
all_admissions_info[5:20]

(58976, 19)


,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG
216,7,118037,0
217,8,159514,0
218,9,150750,1
219,10,184167,0
220,11,194540,0
221,12,112213,1
222,13,143045,0
223,16,103251,0
224,17,194023,0
225,17,161087,0


In [7]:
# 'First' admission if it is a patient's first ever admission
# Exclude patients who die during their 'first' visit

one_admission = pd.read_sql("""SELECT SUBJECT_ID, HOSPITAL_EXPIRE_FLAG FROM admissions
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) = 1
""", conn)
print("Number of people admitted once: " + str(one_admission.shape[0]))
one_admission_deaths = one_admission.iloc[one_admission['HOSPITAL_EXPIRE_FLAG'].values == 1]
one_admission_deaths = one_admission_deaths[['SUBJECT_ID']]
print("People who died on their first admission:")
print(one_admission_deaths.shape[0])
print(one_admission_deaths.head())

one_admission_survivors = one_admission[one_admission['HOSPITAL_EXPIRE_FLAG'] == 0]
one_admission_survivors = one_admission_survivors[['SUBJECT_ID']]
print("People who only had one admission but survived it: " + str(one_admission_survivors.shape[0]))
print(one_admission_survivors[0:15])

mult_admissions = pd.read_sql("""SELECT SUBJECT_ID FROM admissions
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) > 1
ORDER BY CAST(SUBJECT_ID AS UNSIGNED) ASC
""", conn)
print("People who had multiple admissions: " + str(mult_admissions.shape[0]))
print(mult_admissions.head())

# Concatenate all subject IDs from patients who survived their first visit
relevant_patients = pd.concat([one_admission_survivors, mult_admissions], axis = 0)
relevant_patients.columns = relevant_patients.columns.str.strip()
relevant_patients = relevant_patients.sort_values('SUBJECT_ID')
print("Patients who survived their first visit: " + str(relevant_patients.shape[0]))
relevant_patients[0:20]

Number of people admitted once: 38983
People who died on their first admission:
4351
    SUBJECT_ID
7            9
10          12
23          31
43          56
71          91
People who only had one admission but survived it: 34632
    SUBJECT_ID
0            2
1            3
2            4
3            5
4            6
5            7
6            8
8           10
9           11
11          13
12          16
13          18
14          19
15          20
16          22
People who had multiple admissions: 7537
   SUBJECT_ID
0          17
1          21
2          23
3          34
4          36
Patients who survived their first visit: 42169


,SUBJECT_ID
0,2
1,3
2,4
3,5
4,6
5,7
6,8
8,10
9,11
11,13


In [8]:
# All admissions information for relevant patients
all_admissions_for_relevant_patients = pd.merge(relevant_patients, all_admissions_info, how='inner', on='SUBJECT_ID')
print(all_admissions_for_relevant_patients.shape[0])
all_admissions_for_relevant_patients[4:20]

54625


,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG
4,6,107064,0
5,7,118037,0
6,8,159514,0
7,10,184167,0
8,11,194540,0
9,13,143045,0
10,16,103251,0
11,17,194023,0
12,17,161087,0
13,18,188822,0


In [9]:
# Features for all patients
# Only take table info for subjects that are included in our design matrix
# Filter via inner join
admissions_info = pd.read_sql("""SELECT adm.SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME
                                 FROM admissions adm
                                 INNER JOIN patients pat
                                 ON adm.SUBJECT_ID = pat.SUBJECT_ID
                                 ORDER BY pat.SUBJECT_ID ASC
                                 """, conn)
admissions_info['DISCHTIME'] = pd.to_datetime(admissions_info['DISCHTIME'])
admissions_info['ADMITTIME'] = pd.to_datetime(admissions_info['ADMITTIME'])

all_icustays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, OUTTIME, LAST_CAREUNIT, LOS
                              FROM icustays icu
                              INNER JOIN patients pat
                              ON icu.SUBJECT_ID = pat.SUBJECT_ID
                              ORDER BY pat.SUBJECT_ID ASC
                              """, conn)

diagnoses_icd_feats = pd.read_sql("""SELECT diag.SUBJECT_ID, HADM_ID, ICD9_CODE
                                  FROM diagnoses_icd diag
                                  INNER JOIN patients pat
                                  ON diag.SUBJECT_ID = pat.SUBJECT_ID
                                  ORDER BY pat.SUBJECT_ID ASC
                                  """, conn)

# Left joined DRG codes because a lot of DRG severity/mortality data is missing
drgcodes_feats = pd.read_sql("""SELECT drg.SUBJECT_ID, HADM_ID, DRG_SEVERITY, DRG_MORTALITY 
                             FROM drgcodes drg
                             LEFT JOIN patients pat
                             ON drg.SUBJECT_ID = pat.SUBJECT_ID
                             ORDER BY pat.SUBJECT_ID ASC
                             """, conn)

In [10]:
# Further process information for admissions and ICU stays
# Calculate length of stay for each admission
admissions_feats= pd.read_sql("""SELECT adm.SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME
                                 FROM admissions adm
                                 INNER JOIN patients pat
                                 ON adm.SUBJECT_ID = pat.SUBJECT_ID
                                 ORDER BY pat.SUBJECT_ID ASC
                                 """, conn)
# 'HOS_LOS' is length of stay in HOSPITAL, in seconds ('TimedeltaProperties' object only
# has days, seconds, microseconds as attributes)
admissions_feats['HOS_LOS'] = (admissions_info['DISCHTIME'] - admissions_info['ADMITTIME']).dt.seconds
# Only get information from FIRST admission
admissions_feats.sort_values('DISCHTIME')
admissions_feats = admissions_feats.drop_duplicates(subset='SUBJECT_ID', keep='first')
admissions_feats = admissions_feats[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'HOS_LOS']]

# Only get information from LAST ICU stay from an admission
multiple_icustays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, LAST_CAREUNIT, LOS
                                    FROM icustays icu
                                    INNER JOIN patients pat
                                    ON icu.SUBJECT_ID = pat.SUBJECT_ID
                                    GROUP BY HADM_ID
                                    HAVING COUNT(HADM_ID) > 1
                                    ORDER BY icu.SUBJECT_ID ASC
                                    """, conn)
print(multiple_icustays.shape)
multiple_icustays.head()

(3260, 4)


,SUBJECT_ID,HADM_ID,LAST_CAREUNIT,LOS
0,7,118037,NICU,0.2677
1,41,101757,SICU,3.3937
2,68,170467,MICU,3.5368
3,91,121205,CSRU,0.4965
4,109,102024,MICU,2.1826


In [11]:
all_icustays[69:74]

,SUBJECT_ID,HADM_ID,OUTTIME,LAST_CAREUNIT,LOS
69,67,155252,2157-12-02 05:56:38,TSICU,0.2148
70,68,170467,2173-12-19 15:51:16,MICU,3.5368
71,68,170467,2173-12-31 21:33:34,MICU,0.82
72,68,108329,2174-01-14 22:45:42,MICU,6.3983
73,69,190201,2129-04-09 13:41:23,NICU,18.9106


In [12]:
# For each patient, only keep the LAST ICU stay (if there was one)
all_icustays_sorted = all_icustays.sort_values('OUTTIME')
icustays_feats = all_icustays_sorted.drop_duplicates(subset=['HADM_ID'], keep='last')
icustays_feats = icustays_feats.sort_values('SUBJECT_ID')
icustays_feats[67:71]

,SUBJECT_ID,HADM_ID,OUTTIME,LAST_CAREUNIT,LOS
68,67,186474,2155-02-27 15:45:00,SICU,1.7368
72,68,108329,2174-01-14 22:45:42,MICU,6.3983
71,68,170467,2173-12-31 21:33:34,MICU,0.82
73,69,190201,2129-04-09 13:41:23,NICU,18.9106


In [13]:
# Age
pat_adm_df = pd.read_sql("""SELECT pat.SUBJECT_ID, pat.DOB, adm.ADMITTIME
                            FROM patients pat
                            INNER JOIN admissions adm
                            ON pat.SUBJECT_ID = adm.SUBJECT_ID
                            ORDER BY pat.SUBJECT_ID ASC""", conn)

admissions_feats['age'] = (pd.to_datetime(pat_adm_df['ADMITTIME']) - pd.to_datetime(pat_adm_df['DOB'])).dt.days

In [14]:
print(admissions_info.shape)
admissions_info[3:17]

(58976, 4)


,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME
3,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00
5,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00
6,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00
7,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00
8,10,184167,2103-06-28 11:36:00,2103-07-06 12:10:00
9,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00
10,12,112213,2104-08-07 10:15:00,2104-08-20 02:57:00
11,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00
12,16,103251,2178-02-03 06:35:00,2178-02-05 10:51:00


In [15]:
print(admissions_feats.shape[0])
admissions_feats[4:15]

46520


,SUBJECT_ID,HADM_ID,ADMITTIME,HOS_LOS,age
4,6,107064,2175-05-30 07:15:00,31500,24084
5,7,118037,2121-05-23 15:05:00,75120,0
6,8,159514,2117-11-20 10:22:00,14280,0
7,9,150750,2149-11-09 13:06:00,76140,15263
8,10,184167,2103-06-28 11:36:00,2040,0
9,11,194540,2178-04-16 06:18:00,45720,18316
10,12,112213,2104-08-07 10:15:00,60120,26433
11,13,143045,2167-01-08 18:43:00,73920,14560
12,16,103251,2178-02-03 06:35:00,15360,0
13,17,194023,2134-12-27 07:15:00,31800,17332


In [16]:
print(all_icustays.shape)
all_icustays[0:10]

(61532, 5)


,SUBJECT_ID,HADM_ID,OUTTIME,LAST_CAREUNIT,LOS
0,2,163353,2138-07-17 23:32:21,NICU,0.0918
1,3,145834,2101-10-26 20:43:09,MICU,6.0646
2,4,185777,2191-03-17 16:46:31,MICU,1.6785
3,5,178980,2103-02-02 08:06:00,NICU,0.0844
4,6,107064,2175-06-03 13:39:54,SICU,3.6729
5,7,118037,2121-05-23 22:01:00,NICU,0.2677
6,7,118037,2121-05-25 21:10:19,NICU,0.7391
7,8,159514,2117-11-21 14:24:55,NICU,1.0755
8,9,150750,2149-11-14 20:52:14,MICU,5.3231
9,10,184167,2103-07-06 13:51:43,NICU,8.0921


In [17]:
print(diagnoses_icd_feats.shape)
diagnoses_icd_feats.head()

(651047, 3)


,SUBJECT_ID,HADM_ID,ICD9_CODE
0,2,163353,V3001
1,2,163353,V053
2,2,163353,V290
3,3,145834,0389
4,3,145834,78559


In [18]:
print(drgcodes_feats.shape)
drgcodes_feats[15:25]

(125557, 4)


,SUBJECT_ID,HADM_ID,DRG_SEVERITY,DRG_MORTALITY
15,16,103251,,
16,17,194023,1,1
17,17,194023,,
18,17,161087,,
19,17,161087,3,3
20,18,188822,,
21,19,109235,3,3
22,19,109235,,
23,20,157681,,
24,21,111970,,


In [19]:
# Get one overall features matrix for ALL patients
features_for_all_mat = pd.merge(admissions_feats, icustays_feats[['HADM_ID', 'LAST_CAREUNIT', 'LOS']], how='left', on=['HADM_ID'])
print(features_for_all_mat.shape)

# Merge feature dataframe with the patient_cols dataframes so we only keep the features
# for the RELEVANT patients
features_mat = pd.merge(relevant_patients, features_for_all_mat, how='left', on=['SUBJECT_ID'])
print(features_mat.shape)
features_mat[4:20]

(46520, 7)
(42169, 7)


,SUBJECT_ID,HADM_ID,ADMITTIME,HOS_LOS,age,LAST_CAREUNIT,LOS
4,6,107064,2175-05-30 07:15:00,31500,24084,SICU,3.6729
5,7,118037,2121-05-23 15:05:00,75120,0,NICU,0.7391
6,8,159514,2117-11-20 10:22:00,14280,0,NICU,1.0755
7,10,184167,2103-06-28 11:36:00,2040,0,NICU,8.0921
8,11,194540,2178-04-16 06:18:00,45720,18316,SICU,1.5844
9,13,143045,2167-01-08 18:43:00,73920,14560,CSRU,3.666
10,16,103251,2178-02-03 06:35:00,15360,0,NICU,0.0979
11,17,194023,2134-12-27 07:15:00,31800,17332,CSRU,2.071
12,18,188822,2167-10-02 11:18:00,17820,18569,CCU,1.2885
13,19,109235,2108-08-05 16:25:00,68640,-103931,TSICU,1.3017


In [20]:
# !!! Need to generate truth labels for ONLY:
# - patients who survived their first admission
# - the last ICU stay, if there were multiple
# - the first admission, if there were multiple

# Generate additional columns:
# Append truth labels column (Was patient readmitted within 30 days?)
# Append 'ICU stay?' column
subj_adm = all_admissions_for_relevant_patients["SUBJECT_ID"].value_counts()
subj_adm = subj_adm.sort_index()

mult_adm_subj = subj_adm.index[subj_adm > 1]

# Instantiate to all zeros
truth = np.zeros((len(subj_adm)))

subj_adm_idx = list(subj_adm.index)

for subject in mult_adm_subj:
    temp_df = admissions_info[admissions_info["SUBJECT_ID"] == subject]
    # Get time between first discharge and second admission
    first_disc_time = temp_df["DISCHTIME"].iloc[0]
    second_adm_time = temp_df["ADMITTIME"].iloc[1]
    days_to_readm = (pd.to_datetime(second_adm_time) - pd.to_datetime(first_disc_time)).days
    
    # If readmitted within 30 days, set truth to 1
    if days_to_readm <= 30:
        truth[subj_adm_idx.index(subject)] = 1
        
truth_df = pd.DataFrame({'SUBJECT_ID': subj_adm_idx, 'READM': np.array(truth, dtype='int')})
print(truth_df.head())
print(truth_df.shape)

   SUBJECT_ID  READM
0           2      0
1           3      0
2           4      0
3           5      0
4           6      0
(42169, 2)


In [21]:
# Now we have the truth labels, delete the column ADMITTIME from the features_mat
features_mat = features_mat.drop('ADMITTIME', axis=1)

In [22]:
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn import metrics
from sklearn.model_selection import train_test_split

design_mat = features_mat.merge(truth_df,how='inner',on="SUBJECT_ID")
X = design_mat.iloc[:,2:-1]

LCU = design_mat["LAST_CAREUNIT"]
le = LabelEncoder() # categorical encoder
LCU = le.fit_transform(LCU.astype(str))
X['LAST_CAREUNIT'] = LCU
X['LOS'] = pd.to_numeric(X['LOS']).fillna(0)

# normalize feature values
for i in list(X):
    X[i] = normalize(X[i][:].values.reshape(-1,1),axis=0)
    
# make train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, design_mat.iloc[:,-1],test_size = 0.33, random_state = 0)

print(X.shape)
X.head()

(42169, 4)


,HOS_LOS,age,LAST_CAREUNIT,LOS
0,0.006924,0.000000,0.005092,0.000038
1,0.006290,0.004749,0.003395,0.002530
2,0.006084,0.002969,0.003395,0.000700
3,0.002583,0.000000,0.005092,0.000035
4,0.002922,0.004092,0.006790,0.001532


In [23]:
print(design_mat.shape)
print(features_mat.shape)

(42169, 7)
(42169, 6)


### Classifiers

### Metrics for evaluating models

In [24]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf=clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
AUC = metrics.auc(fpr, tpr)

/Users/kevinmchu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [25]:
AUC

0.5289431979314371

In [34]:
# Create TRUTH.csv file
truth_and_hadm_df = truth_df.merge(features_mat.loc[:, ["SUBJECT_ID", "HADM_ID"]])
truth_and_hadm_array = truth_and_hadm_df.get_values()

# Save
np.savetxt("TRUTH.csv", truth_and_hadm_array, fmt='%d', delimiter=',')